In [1]:
import pandas as pd
import postgres_helper

### Create Connection

In [2]:
pg_conn = postgres_helper.create_postgres_connection()
pg_conn

<connection object at 0x7fa9c933be10; dsn: 'user=cjpskuee password=xxx dbname=cjpskuee host=salt.db.elephantsql.com', closed: 0>

#### Top 10 rows from Titanic

In [3]:
query = 'SELECT * FROM titanic LIMIT 10;'
postgres_helper.select_query(pg_conn, query)

,passenger_id,survived,pclass,name,sex,age,siblings_spouses,parents_children,fare
0,0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,5,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,6,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,9,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708


#### 1. How many passengers survived, and how many died?

In [4]:
query = '''SELECT survived, COUNT(survived)
FROM titanic
GROUP BY survived;'''
postgres_helper.select_query(pg_conn, query)

,survived,count
0,0,545
1,1,342


#### 2. How many passengers were in each class?

In [5]:
query = '''SELECT pclass, COUNT(pclass)
FROM titanic
GROUP BY pclass;'''
postgres_helper.select_query(pg_conn, query)

,pclass,count
0,1,216
1,3,487
2,2,184


#### 3. How many passengers survived/died within each class?

In [6]:
query = '''SELECT pclass,survived, COUNT(survived)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;;'''
postgres_helper.select_query(pg_conn, query)

,pclass,survived,count
0,1,0,80
1,1,1,136
2,2,0,97
3,2,1,87
4,3,0,368
5,3,1,119


#### 4. What was the average age of each passenger class?

In [7]:
query = '''SELECT pclass, AVG(age)
FROM titanic
GROUP BY pclass;'''
postgres_helper.select_query(pg_conn, query)

,pclass,avg
0,1,38.788981
1,3,25.188747
2,2,29.868641


#### 5. What was the average fare by passenger class? By survival?

In [8]:
query = '''SELECT pclass, survived, AVG(fare)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;'''
postgres_helper.select_query(pg_conn, query)

,pclass,survived,avg
0,1,0,64.684007
1,1,1,95.608029
2,2,0,19.412328
3,2,1,22.055700
4,3,0,13.711853
5,3,1,13.694887


#### 5. How many siblings/spouses aboard on average, by passenger class? By survival?

In [9]:
query = '''SELECT pclass, AVG(siblings_spouses)
FROM titanic
GROUP BY pclass
ORDER BY pclass ASC;'''
postgres_helper.select_query(pg_conn, query)

,pclass,avg
0,1,0.416667
1,2,0.402174
2,3,0.620123


In [10]:
query = '''SELECT survived, AVG(siblings_spouses)
FROM titanic
GROUP BY survived
ORDER BY survived ASC;'''
postgres_helper.select_query(pg_conn, query)

,survived,avg
0,0,0.557798
1,1,0.473684


In [11]:
query = '''SELECT pclass, survived, AVG(siblings_spouses)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;'''
postgres_helper.select_query(pg_conn, query)

,pclass,survived,avg
0,1,0,0.287500
1,1,1,0.492647
2,2,0,0.319588
3,2,1,0.494253
4,3,0,0.679348
5,3,1,0.436975


#### 6. How many parents/children aboard on average, by passenger class? By survival?

In [12]:
query = '''SELECT pclass, AVG(parents_children)
FROM titanic
GROUP BY pclass
ORDER BY pclass ASC;
'''
postgres_helper.select_query(pg_conn, query)

,pclass,avg
0,1,0.356481
1,2,0.380435
2,3,0.396304


In [13]:
query = '''SELECT survived, AVG(parents_children)
FROM titanic
GROUP BY survived
ORDER BY survived ASC;
'''
postgres_helper.select_query(pg_conn, query)

,survived,avg
0,0,0.332110
1,1,0.464912


In [14]:
query = '''SELECT pclass, survived, AVG(parents_children)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;
'''
postgres_helper.select_query(pg_conn, query)

,pclass,survived,avg
0,1,0,0.300000
1,1,1,0.389706
2,2,0,0.144330
3,2,1,0.643678
4,3,0,0.388587
5,3,1,0.420168


#### 7. Do any passengers have the same name?

In [15]:
query = '''SELECT name FROM titanic WHERE name NOT IN (SELECT DISTINCT name FROM titanic);
'''
postgres_helper.select_query(pg_conn, query)

,name


In [16]:
query = '''SELECT (SELECT COUNT(name) FROM titanic) - (SELECT COUNT(DISTINCT name)
FROM titanic) AS name;
'''
postgres_helper.select_query(pg_conn, query)

,name
0,0


#### 8. (Bonus! Hard, may require pulling and processing with Python) How many married couples were aboard the Titanic? Assume that two people (one Mr. and one Mrs.) with the same last name and with at least 1 sibling/spouse aboard are a married couple.

In [17]:
query = 'SELECT name FROM titanic;'
df = postgres_helper.select_query(pg_conn, query)
df.head(20)

,name
0,Mr. Owen Harris Braund
1,Mrs. John Bradley (Florence Briggs Thayer) Cum...
2,Miss. Laina Heikkinen
3,Mrs. Jacques Heath (Lily May Peel) Futrelle
4,Mr. William Henry Allen
5,Mr. James Moran
6,Mr. Timothy J McCarthy
7,Master. Gosta Leonard Palsson
8,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson
9,Mrs. Nicholas (Adele Achem) Nasser


Look over this Chris Albon blog: https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/

In [18]:
%%writefile titanic_queries_postgre.py

# Make a Connection
pg_conn = postgres_helper.create_postgres_connection(verbose=True)

# List top 10 titanic rows
query = 'SELECT * FROM titanic LIMIT 10;'
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

# How many passengers were in each class?
query = '''SELECT pclass, COUNT(pclass)
FROM titanic
GROUP BY pclass;'''
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

# How many passengers survived/died within each class?
query = '''SELECT pclass, survived, COUNT(survived)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;;'''
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

# What was the average age of each passenger class?
query = '''SELECT pclass, AVG(age)
FROM titanic
GROUP BY pclass;'''
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

# What was the average fare by passenger class? By survival?
query = '''SELECT pclass, survived, AVG(fare)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;'''
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

# How many siblings/spouses aboard on average, by passenger class? By survival?
query = '''SELECT pclass, survived, AVG(siblings_spouses)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;'''
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

# How many parents/children aboard on average, by passenger class? By survival?
query = '''SELECT pclass, survived, AVG(parents_children)
FROM titanic
GROUP BY pclass, survived
ORDER BY pclass, survived ASC;
'''
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

# Do any passengers have the same name?
query = '''SELECT name FROM titanic WHERE name NOT IN (SELECT DISTINCT name FROM titanic);
'''
results = postgres_helper.select_query(pg_conn, query)
print(results)
print('-'*80)
print('\n')

Writing titanic_queries_postgre.py
